In [1]:
import matplotlib
import pandas as pd
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.naive_bayes import *
from sklearn.linear_model import LinearRegression
from re import sub
def mdy_to_ymd(d):
    return time.mktime(datetime.strptime(d, '%d-%b-%y').timetuple())

class Beautify:

    @staticmethod
    def beautify(path):
        dataset = pd.read_csv(path)
        dataset.columns = ["date", "price", "open", "high", "low", "vol", "change"]
        dataset["date"] = dataset.apply(lambda row: mdy_to_ymd(row['date']), axis=1)
        for col in ["price", "open", "high", "low", "vol", "change"]:
            dataset[col] = dataset.apply(
                lambda row: (lambda x: 0 if x == '-' else float(x))(sub(r'[^\d.-]', '', row[col])),
                axis=1)
        dataset = dataset.sort_values(["date"])
        return dataset


pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [2]:
from sklearn.ensemble import BaggingRegressor

In [3]:
dataset = Beautify.beautify("dataset.csv")

dataset = dataset[["date","price","change"]]
dataset["change"] = dataset.apply(lambda x: -1 if x["change"] <= 0 else 1,axis=1)

In [4]:
for i in [1,2,3,5,7,10,12,15,20,30,45]:
    dataset[f"s_{i}"] = dataset["price"].shift(i,fill_value=0)

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

In [6]:
TRAIN = time.mktime(datetime.strptime("20-01-02", '%y-%m-%d').timetuple())
train = dataset[dataset.date < TRAIN]
test = dataset[dataset.date >= TRAIN]

In [7]:
train.drop("date",axis=1,inplace=True)
test.drop("date",axis=1,inplace=True)

E:\Anaconda2021\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [78]:
from sklearn.ensemble import BaggingRegressor,AdaBoostRegressor,VotingRegressor,GradientBoostingRegressor,RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB

In [25]:
cls = VotingRegressor(estimators=[("l",Lasso()),("lr",LinearRegression()),("knn",KNeighborsRegressor())])
pred = cls.fit(train.drop("price",axis=1), train.price).predict(test.drop("price",axis=1))
res = pd.DataFrame({"index": np.arange(len(pred)), "real": test.price, "pred": pred})

E:\Anaconda2021\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 60206971.72609579, tolerance: 4106899.6988909775
  model = cd_fast.enet_coordinate_descent(


In [26]:
print(res[(res.pred * 0.95 <= res.real) & (res.pred * 1.05 >= res.real)].shape[0] / res.shape[0] * 100)
from sklearn.metrics import mean_squared_error
mean_squared_error(res.pred,res.real)

53.90946502057613


35600915.04876527

In [31]:
cls = GradientBoostingRegressor()
pred = cls.fit(train.drop("price",axis=1), train.price).predict(test.drop("price",axis=1))
res = pd.DataFrame({"index": np.arange(len(pred)), "real": test.price, "pred": pred})
print(res[(res.pred * 0.95 <= res.real) & (res.pred * 1.05 >= res.real)].shape[0] / res.shape[0] * 100)
from sklearn.metrics import mean_squared_error
mean_squared_error(res.pred,res.real)

62.757201646090536


264894496.92937002

In [40]:
for m in (models:=[Lasso(),LinearRegression(),KNeighborsRegressor()]):
    print(str(m))
    cls = BaggingRegressor(base_estimator=m,n_estimators=10)
    cls.estimators_ = models.remove(m)
    pred = cls.fit(train.drop("price",axis=1), train.price).predict(test.drop("price",axis=1))
    res = pd.DataFrame({"index": np.arange(len(pred)), "real": test.price, "pred": pred})
    print(res[(res.pred * 0.95 <= res.real) & (res.pred * 1.05 >= res.real)].shape[0] / res.shape[0] * 100)
    from sklearn.metrics import mean_squared_error
    print(mean_squared_error(res.pred,res.real))

Lasso()


E:\Anaconda2021\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 54222360.297070935, tolerance: 4063701.203863298
  model = cd_fast.enet_coordinate_descent(
E:\Anaconda2021\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 57602918.93519484, tolerance: 4243445.3897987185
  model = cd_fast.enet_coordinate_descent(
E:\Anaconda2021\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 55984541.77357052, tolerance: 4163083.358879959
  model = cd_fast.enet_coordinate_descent(
E:\Anaconda2021\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You

87.24279835390946
1088563.8905726306
LinearRegression()
87.24279835390946
1091416.723207294
KNeighborsRegressor()
23.662551440329217
303233859.699593


In [77]:
cls = AdaBoostRegressor(base_estimator=LinearRegression(),n_estimators=100)
pred = cls.fit(train.drop("price",axis=1), train.price).predict(test.drop("price",axis=1))
res = pd.DataFrame({"index": np.arange(len(pred)), "real": test.price, "pred": pred})
print("accuracy:",res[(res.pred * 0.95 <= res.real) & (res.pred * 1.05 >= res.real)].shape[0] / res.shape[0] * 100)
from sklearn.metrics import mean_squared_error
print("mse:",mean_squared_error(res.pred,res.real))

accuracy: 89.30041152263375
mse: 1035213.6440181762


In [95]:
cls = RandomForestRegressor(bootstrap=False)
pred = cls.fit(train.drop("price",axis=1), train.price).predict(test.drop("price",axis=1))
res = pd.DataFrame({"index": np.arange(len(pred)), "real": test.price, "pred": pred})
print("accuracy:",res[(res.pred * 0.95 <= res.real) & (res.pred * 1.05 >= res.real)].shape[0] / res.shape[0] * 100)
from sklearn.metrics import mean_squared_error
print("mse:",mean_squared_error(res.pred,res.real))

accuracy: 53.49794238683128
mse: 262158256.59917554
